In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima.model import ARIMA
# from dart import ts_spikes
from statsmodels.stats.diagnostic import het_arch
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.seasonal import STL
from scipy.optimize import curve_fit

In [15]:
GRID_SIZE = 50
NUMBER = 1000
BATCH_SIZE = 25

In [16]:
latitude = np.repeat(range(GRID_SIZE),GRID_SIZE * NUMBER)
longitude = np.tile(np.arange(GRID_SIZE),GRID_SIZE * NUMBER) 
power = np.random.random(GRID_SIZE*GRID_SIZE*NUMBER)
df = pd.DataFrame({'latitude' : latitude, 'longitude' : longitude, 'power' :  power})
df = df.sort_values(by = ['latitude', 'longitude'])
df = df.reset_index(drop=True)
df['spatial_group'] = df.index // NUMBER
df['temporal_group'] = np.tile(np.arange(NUMBER) // BATCH_SIZE, GRID_SIZE * GRID_SIZE)

In [17]:
df.groupby(['temporal_group', 'spatial_group']).apply(lambda group: acf(group['power'], nlags=10))

temporal_group  spatial_group
0               0                [1.0, 0.010419585873241231, 0.0533207523414476...
                1                [1.0, 0.2089137547737358, 0.08460588489754058,...
                2                [1.0, 0.0618009460925382, -0.01171485422847017...
                3                [1.0, -0.1993879828391588, -0.0108133345005514...
                4                [1.0, 0.16981562021490923, 0.12905340843764562...
                                                       ...                        
39              2495             [1.0, -0.08842281363393534, -0.108417071027382...
                2496             [1.0, -0.31439524431230803, 0.1669828494750995...
                2497             [1.0, -0.2291141360491174, -0.2411010239200786...
                2498             [1.0, -0.199784600260765, -0.06747884476746367...
                2499             [1.0, 0.20766301882547722, 0.10372497263962041...
Length: 100000, dtype: object

In [22]:
_,group = next(iter(df.groupby(['temporal_group', 'spatial_group'])))

In [88]:
# Different aggregate functions
def difference_time_series(x):
    ''''''
    return x.diff().dropna()


def x_acf1(x):
    '''The autocorrelation function of the series'''
    return acf(x, nlags=1)[1]


def diff1_acf1(x):
    '''The autocorrelation function of the differenced series'''
    first_difference = difference_time_series(x)
    return acf(first_difference, nlags=1)[1]


def diff2_acf1(x):
    '''The autocorrelation function of the second-order differenced series'''
    first_difference = difference_time_series(x)
    second_difference = difference_time_series(first_difference)
    return acf(second_difference, nlags=1)[1]


def e_acf1(x):
    '''The autocorrelation function of the residuals'''
    return acf(data - x.rolling(window=2).mean().dropna(), nlags=1)[1]


def x_acf10(x):
    '''The sum of squares of the first 10 autocorrelation coefficients for series'''
    return sum(acf(group['power'], nlags=10)**2)

def diff1_acf10(x):
    '''The sum of squares of the first 10 autocorrelation coefficients for the differenced series'''
    first_difference = difference_time_series(x)
    return sum(acf(first_difference['power'], nlags=10)**2)

def diff2_acf10(x):
    '''The sum of squares of the first 10 autocorrelation coefficients for second-order the differenced series'''
    x = difference_time_series(x)
    return sum(acf(group['power'], nlags=10)**2)


def e_acf10(x,window_size=2):
    '''The sum of squares of the first 10 autocorrelation coefficients of the residuals'''
    # Calculate the rolling mean
    rolling_mean = data_pd.rolling(window=window_size).mean().dropna()

    # Calculate the residuals by subtracting the rolling mean from the original data
    residuals = data[:-window_size+1] - rolling_mean.values

    # Compute the autocorrelation function for the residuals
    e_acf = acf(residuals, nlags=10)

    # Calculate the sum of squares of the first 10 autocorrelation coefficients
    e_acf10 = np.sum(np.square(e_acf[:10]))

    return e_acf10


def entropy(x):
    '''The spectral entropy is the Shannon entropy''''
    return -np.sum(x * np.log2(x))


def crossing_points(x):
    '''The number of times the temporal data-set crosses the median line'''
    return len(np.where(np.diff((x['power'] > x['power'].median())))[0])


def flat_spots(x):
    '''The temporal dataset is divided into ten equally blocks then the largest runlength represents the value of flat_spots'''
    pass


def nonlinear(x):
    '''The nonlinearity is estimated from a modified Teräsvirta’s test'''
    pass


def linearity(x):
    ''' The strength of curvature are estimated from the coefficients of the orthogonal quadratic regression.'''
    pass


def curvature(x):
    ''''''
    pass


def x_pacf5(x):
    '''Thee sum of the first 5 partial autocorrelation coefficients of the series''''
    return np.sum(pacf(x, nlags=5)[:5])


def diff1_pacf5(x):
    ''''The sum of the first 5 partial autocorrelation coefficients of the differenced series'''
    first_difference = difference_time_series(x)
    return np.sum(pacf(first_difference, nlags=5)[:5])
    

def diff2_pacf5(x):
    '''The sum of the first 5 partial autocorrelation coefficients of the second-order differenced series'''
    first_difference = difference_time_series(x)
    second_difference = difference_time_series(first_difference)
    return np.sum(pacf(second_difference, nlags=5)[:5])


def lumpiness(x):
    '''Temporal dataset is divided into non-overlapping windoes the variance of the mean of the tiled windows'''
    pass


def stability(x):
    '''Temporal dataset is divided into non-overlapping windoes the variance of the variance of the tiled windows'''
    pass


def arch_stat(x):
    '''Stability based on Lagrange Multiplier Test'''
    lag, arch_stat, p_value, f_test = het_arch(data, maxlag=10)
    return arch_stat


def trend(x):
    '''The strength of the trend is found from Seasonal-Trend decomposition using LOESS'''
    stl = STL(x, period=12)
    res = stl.fit()

    # calculate trend strength measure
    trend_var, total_var = np.var(res.trend), np.var(x)
    return trend_var / total_var


def spike(x):
    '''Variance of the leave-one-out variances of the residuals'''
    model = ARIMA(x, order=(1,1,1))
    model_fit = model.fit()

    # calculate residuals
    residuals = pd.Series(model_fit.resid)

    # calculate variance of residuals
    residuals_var = np.var(residuals)

    # calculate leave-one-out variances of residuals
    loocv_variances = []
    for i in range(len(residuals)):
        # remove one observation from the data
        data_loo = x.drop(data.index[i])
        # fit ARIMA model to the leave-one-out data
        model_loo = ARIMA(data_loo, order=(1,1,1))
        model_fit_loo = model_loo.fit()
        # calculate residuals of the leave-one-out model
        residuals_loo = pd.Series(model_fit_loo.resid)
        # calculate variance of residuals of the leave-one-out model
        residuals_var_loo = np.var(residuals_loo)
        # add variance to the list
        loocv_variances.append(residuals_var_loo)

    # calculate spike as the variance of the leave-one-out variances of residuals
    spike = np.var(loocv_variances)
